# Quantitative trading in China A stock market with FinRL

In [1]:
!pip install stockstats
!pip install tushare


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install TA-Lib


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install gym


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
!pwd

/Users/zhengshuang/Documents/ProgramingGuide/SourceCode/GithubRepo/everyfine/FinRLLearn/FinRL-Tutorials-Learn/1-Introduction


In [5]:
# !git clone https://github.com/AI4Finance-Foundation/FinRL-Meta
%cd FinRL-Meta

/Users/zhengshuang/Documents/ProgramingGuide/SourceCode/GithubRepo/everyfine/FinRLLearn/FinRL-Tutorials-Learn/1-Introduction/FinRL-Meta


In [6]:
!pwd

/Users/zhengshuang/Documents/ProgramingGuide/SourceCode/GithubRepo/everyfine/FinRLLearn/FinRL-Tutorials-Learn/1-Introduction/FinRL-Meta


## Import Modules

In [6]:
import warnings

warnings.filterwarnings("ignore")
import pandas as pd
from IPython import display

display.set_matplotlib_formats("svg")

from meta import config
from meta.data_processor import DataProcessor
from main import check_and_make_directories

In [7]:
from meta.data_processors.tushare import Tushare, ReturnPlotter
from meta.env_stock_trading.env_stocktrading_China_A_shares import StockTradingEnv
from agents.stablebaselines3_models import DRLAgent

In [8]:
import os
from typing import List
from argparse import ArgumentParser
from meta.config_tickers import DOW_30_TICKER 
from meta.config import ( DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR, INDICATORS, TRAIN_START_DATE, TRAIN_END_DATE, TEST_START_DATE, TEST_END_DATE, TRADE_START_DATE, TRADE_END_DATE, ERL_PARAMS, RLlib_PARAMS, SAC_PARAMS, ALPACA_API_KEY, ALPACA_API_SECRET, ALPACA_API_BASE_URL, )

In [9]:
import pyfolio
from pyfolio import timeseries
pd.options.display.max_columns = None
print("All Modules have been imported")

All Modules have been imported


## Create Folders

In [11]:
import os
''' 
use check_and_make_directories() to replace the following

if not os.path.exists("./datasets"): 
  os.makedirs("./datasets") 
if not os.path.exists("./trained_models"): 
  os.makedirs("./trained_models") 
if not os.path.exists("./tensorboard_log"): 
  os.makedirs("./tensorboard_log") 
if not os.path.exists("./results"): 
  os.makedirs("./results") 
'''
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

## Download data, cleaning and feature engineering

- 600000.SH 浦发银行
- 600009.SH 上海机场
- 600016.SH 民生银行
- 600028.SH 中国石化
- 600030.SH 中信证券
- 600031.SH 三一重工
- 600036.SH 招商银行
- 600050.SH 中国联通
- 600104.SH 上汽集团
- 600196.SH 复星医药
- 600276.SH 恒瑞医药
- 600309.SH 万华化学
- 600519.SH 贵州茅台
- 600547.SH 山东黄金
- 600570.SH 恒生电子


In [15]:
ticker_list = ['600000.SH', '600009.SH', '600016.SH', '600028.SH', '600030.SH', '600031.SH', '600036.SH', '600050.SH', '600104.SH', '600196.SH', '600276.SH', '600309.SH', '600519.SH', '600547.SH', '600570.SH']

TRAIN_START_DATE = '2015-01-01' 
TRAIN_END_DATE= '2019-08-01' 
TRADE_START_DATE = '2019-08-01' 
TRADE_END_DATE = '2020-01-03'

In [75]:
TIME_INTERVAL = '1d'
kwargs = {}
kwargs['token'] = '27080ec403c0218f96f388bca1b1d85329d563c91a43672239619ef5' 
p = DataProcessor(data_source='tushare', start_date=TRAIN_START_DATE, end_date=TRADE_END_DATE,time_interval=TIME_INTERVAL, **kwargs)

tushare successfully connected


### Download and Clean

In [17]:
p.download_data(ticker_list=ticker_list)
p.clean_data()
p.fillna()

100%|██████████| 15/15 [00:05<00:00,  2.90it/s]


Download complete! Dataset saved to ./data/dataset.csv. 
Shape of DataFrame: (17960, 8)
Shape of DataFrame:  (18315, 8)


### Add technical indicator

In [18]:
p.add_technical_indicator(config.INDICATORS)
p.fillna()

tech_indicator_list:  ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma']
indicator:  macd
indicator:  boll_ub
indicator:  boll_lb
indicator:  rsi_30
indicator:  cci_30
indicator:  dx_30
indicator:  close_30_sma
indicator:  close_60_sma
Succesfully add technical indicators
Shape of DataFrame:  (18270, 17)


## Split training dataset

In [19]:
train = p.data_split(p.dataframe, TRAIN_START_DATE, TRAIN_END_DATE)
print(f"train.tic.unique(): {train.tic.unique()}")
print(f"len(train.tic.unique()): {len(train.tic.unique())}")

train.tic.unique(): ['600000.SH' '600009.SH' '600016.SH' '600028.SH' '600030.SH' '600031.SH'
 '600036.SH' '600050.SH' '600104.SH' '600196.SH' '600276.SH' '600309.SH'
 '600519.SH' '600547.SH' '600570.SH']
len(train.tic.unique()): 15


In [20]:
train.head()

,tic,time,index,open,high,low,close,adjusted_close,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,600000.SH,2015-01-08,45,15.87,15.88,15.20,15.25,15.25,3306271.72,-0.032571,16.617911,15.012089,6.058641,-125.593009,23.014040,15.8150,15.8150
0,600009.SH,2015-01-08,46,20.18,20.18,19.73,20.00,20.00,198117.45,-0.016008,20.663897,19.736103,12.828915,-90.842491,100.000000,20.2000,20.2000
0,600016.SH,2015-01-08,47,10.61,10.66,10.09,10.20,10.20,4851684.17,-0.018247,10.957604,9.997396,11.862558,-99.887006,100.000000,10.4775,10.4775
0,600028.SH,2015-01-08,48,7.09,7.41,6.83,6.85,6.85,8190902.35,-0.008227,7.342000,6.743000,27.409248,36.578171,64.934862,7.0425,7.0425
0,600030.SH,2015-01-08,49,36.40,36.70,34.68,35.25,35.25,6376268.69,0.032910,36.576444,33.808556,61.517448,47.947020,100.000000,35.1925,35.1925


In [21]:
train.shape

(16695, 17)

In [22]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension * (len(config.INDICATORS) +2) +1
print(f"stock_dimension: {stock_dimension}, state_space: {state_space}")

stock_dimension: 15, state_space: 151


## Error: AttributeError: 'DataFrame' object has no attribute 'date'
在akshare获取的数据中并不包含date列，需要将time列转换为date

In [23]:
train = train.rename(columns={'time': 'date'})
train.head()

,tic,date,index,open,high,low,close,adjusted_close,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,600000.SH,2015-01-08,45,15.87,15.88,15.20,15.25,15.25,3306271.72,-0.032571,16.617911,15.012089,6.058641,-125.593009,23.014040,15.8150,15.8150
0,600009.SH,2015-01-08,46,20.18,20.18,19.73,20.00,20.00,198117.45,-0.016008,20.663897,19.736103,12.828915,-90.842491,100.000000,20.2000,20.2000
0,600016.SH,2015-01-08,47,10.61,10.66,10.09,10.20,10.20,4851684.17,-0.018247,10.957604,9.997396,11.862558,-99.887006,100.000000,10.4775,10.4775
0,600028.SH,2015-01-08,48,7.09,7.41,6.83,6.85,6.85,8190902.35,-0.008227,7.342000,6.743000,27.409248,36.578171,64.934862,7.0425,7.0425
0,600030.SH,2015-01-08,49,36.40,36.70,34.68,35.25,35.25,6376268.69,0.032910,36.576444,33.808556,61.517448,47.947020,100.000000,35.1925,35.1925


In [21]:
## Train

In [24]:
env_kwargs = {
    "stock_dim": stock_dimension, 
    "hmax": 1000, 
    "initial_amount": 1000000, 
    "buy_cost_pct": 6.87e-5, 
    "sell_cost_pct": 1.0687e-3, 
    "reward_scaling": 1e-4, 
    "state_space": state_space, 
    "action_space": stock_dimension, 
    "tech_indicator_list": config.INDICATORS, 
    "print_verbosity": 1, 
    "initial_buy": True, 
    "hundred_each_trade": True
}

e_train_gym = StockTradingEnv(df=train, **env_kwargs)

In [25]:
env_train, _ = e_train_gym.get_sb_env()
print(f"type(env_train): {print(type(env_train))}")

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>
type(env_train): None


In [24]:
!pip install 'shimmy>=2.0'


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


### DDPG

In [26]:
agent = DRLAgent(env= env_train)
DDPG_PARAMS = { 
    "batch_size": 256, 
    "buffer_size": 50000, 
    "learning_rate": 0.0005, 
    "action_noise": "normal",
} 
POLICY_KWARGS = dict(net_arch=dict(pi=[64, 64], qf=[400, 300])) 

model_ddpg = agent.get_model('ddpg', model_kwargs=DDPG_PARAMS, policy_kwargs=POLICY_KWARGS)
trained_ddpg = agent.train_model(model=model_ddpg,tb_log_name='ddpg',total_timesteps=100000)
trained_ddpg.save(TRAIN_START_DATE+'/ashare_trained_models_ddpg')

{'batch_size': 256, 'buffer_size': 50000, 'learning_rate': 0.0005, 'action_noise': NormalActionNoise(mu=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], sigma=[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1])}
Using cpu device
Logging to tensorboard_log/ddpg/ddpg_3
Episode: 2
day: 1112, episode: 2
begin_total_asset: 1000000.00
end_total_asset: 1033393.15
total_reward: 33393.15
total_cost: 13683.85
total_trades: 1523
Sharpe: 0.183
Episode: 3
day: 1112, episode: 3
begin_total_asset: 1000000.00
end_total_asset: 887956.42
total_reward: -112043.58
total_cost: 422.58
total_trades: 79
Sharpe: 0.059
Episode: 4
day: 1112, episode: 4
begin_total_asset: 1000000.00
end_total_asset: 885661.13
total_reward: -114338.87
total_cost: 422.87
total_trades: 80
Sharpe: 0.057
Episode: 5
day: 1112, episode: 5
begin_total_asset: 1000000.00
end_total_asset: 879743.08
total_reward: -120256.92
total_cost: 422.92
total_trades: 81
Sharpe: 0.049
-----------------------------------
| time/              | 

### A2C

In [27]:
agent = DRLAgent(env= env_train)
model_a2c = agent.get_model('a2c')

trained_a2c = agent.train_model(model=model_a2c, tb_log_name='a2c',total_timesteps=50000)
trained_a2c.save(TRAIN_START_DATE+'/ashare_trained_models_a2c')

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device
Logging to tensorboard_log/a2c/a2c_3
---------------------------------------
| time/                 |             |
|    fps                | 599         |
|    iterations         | 100         |
|    time_elapsed       | 0           |
|    total_timesteps    | 500         |
| train/                |             |
|    entropy_loss       | -21.4       |
|    explained_variance | -77.4       |
|    learning_rate      | 0.0007      |
|    n_updates          | 99          |
|    policy_loss        | -7.33       |
|    reward             | -0.03825828 |
|    std                | 1.01        |
|    value_loss         | 0.158       |
---------------------------------------
----------------------------------------
| time/                 |              |
|    fps                | 605          |
|    iterations         | 200          |
|    time_elapsed       | 1            |
|    total_timesteps    | 1000         |
| 

### Trade

In [29]:
trade = p.data_split(p.dataframe, TRADE_START_DATE, TRADE_END_DATE) 
trade = trade.rename(columns={'time': 'date'})
trade.head()
env_kwargs = { 
    "stock_dim": stock_dimension, 
    "hmax": 1000, 
    "initial_amount": 1000000, 
    "buy_cost_pct": 6.87e-5, 
    "sell_cost_pct": 1.0687e-3, 
    "reward_scaling": 1e-4, 
    "state_space": state_space, 
    "action_space": stock_dimension, 
    "tech_indicator_list": config.INDICATORS, 
    "print_verbosity": 1, 
    "initial_buy": False, 
    "hundred_each_trade": True 
} 
e_trade_gym = StockTradingEnv(df=trade, **env_kwargs)

In [28]:
trade

,tic,date,index,open,high,low,close,adjusted_close,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,600000.SH,2019-08-01,16740,11.76,11.85,11.63,11.65,11.65,324717.02,0.061723,11.954291,11.217709,51.629338,30.105973,9.109642,11.632667,11.516667
0,600009.SH,2019-08-01,16741,82.78,83.46,81.60,82.01,82.01,72583.66,1.456486,85.100887,78.654113,57.259597,28.697822,13.903367,81.707000,76.533333
0,600016.SH,2019-08-01,16742,6.04,6.06,6.01,6.02,6.02,517116.83,-0.045649,6.088116,5.935884,43.258474,-43.433030,5.774779,6.139667,6.162500
0,600028.SH,2019-08-01,16743,5.25,5.27,5.21,5.23,5.23,616872.58,-0.030634,5.362514,5.179486,42.928615,-77.147867,22.469955,5.324333,5.357333
0,600030.SH,2019-08-01,16744,23.09,23.19,22.81,22.92,22.92,809710.56,0.143135,23.703768,22.720232,51.752238,-102.582689,11.905549,23.385667,21.918333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,600276.SH,2020-01-02,18295,88.00,88.10,86.85,87.68,87.68,210809.91,-0.345742,87.726881,82.918119,53.727892,55.084385,2.193577,86.140333,87.454167
103,600309.SH,2020-01-02,18296,56.50,57.60,55.69,55.89,55.89,213379.65,1.824002,56.464963,49.288037,67.318349,154.362552,50.898752,51.418667,48.306667
103,600519.SH,2020-01-02,18297,1128.00,1145.06,1116.00,1130.00,1130.00,148099.16,-3.944016,1188.194602,1121.743398,46.814398,-98.839848,24.496850,1158.660667,1175.338000
103,600547.SH,2020-01-02,18298,32.31,32.96,32.26,32.90,32.90,489106.37,0.348359,32.722690,30.612310,52.089075,155.084385,32.165604,31.288667,31.498333


In [30]:
from stable_baselines3 import DDPG

trained_model = DDPG.load(TRAIN_START_DATE+'/ashare_trained_models_ddpg')

In [31]:
df_account_value, df_actions = DRLAgent.DRL_prediction(model=trained_model, environment=e_trade_gym)
df_account_value
# df_actions

Episode: 2
day: 103, episode: 2
begin_total_asset: 1000000.00
end_total_asset: 1095681.31
total_reward: 95681.31
total_cost: 68.69
total_trades: 63
Sharpe: 1.439
hit end!


,date,account_value
0,2019-08-01,9.999955e+05
1,2019-08-02,9.974511e+05
2,2019-08-05,9.943469e+05
3,2019-08-06,9.938627e+05
4,2019-08-07,9.912585e+05
...,...,...
98,2019-12-25,1.050891e+06
99,2019-12-26,1.063477e+06
100,2019-12-27,1.056715e+06
101,2019-12-30,1.094123e+06


In [32]:
df_actions.to_csv("ddpg_action.csv", index=False)
df_actions

,600000.SH,600009.SH,600016.SH,600028.SH,600030.SH,600031.SH,600036.SH,600050.SH,600104.SH,600196.SH,600276.SH,600309.SH,600519.SH,600547.SH,600570.SH
date,,,,,,,,,,,,,,,
2019-08-01,1000,0,0,1000,1000,0,0,0,0,1000,0,0,0,0,0
2019-08-01,1000,0,0,1000,1000,0,0,0,0,1000,0,0,0,0,0
2019-08-02,1000,0,0,1000,1000,0,0,0,0,1000,0,0,0,0,0
2019-08-05,1000,0,0,1000,1000,0,0,0,0,1000,0,0,0,0,0
2019-08-06,1000,0,0,1000,1000,0,0,0,0,1000,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2019-12-25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2019-12-26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Backtest

### matplotlib inline

In [31]:
trade

,tic,date,index,open,high,low,close,adjusted_close,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,600000.SH,2019-08-01,16740,11.76,11.85,11.63,11.65,11.65,324717.02,0.061723,11.954291,11.217709,51.629338,30.105973,9.109642,11.632667,11.516667
0,600009.SH,2019-08-01,16741,82.78,83.46,81.60,82.01,82.01,72583.66,1.456486,85.100887,78.654113,57.259597,28.697822,13.903367,81.707000,76.533333
0,600016.SH,2019-08-01,16742,6.04,6.06,6.01,6.02,6.02,517116.83,-0.045649,6.088116,5.935884,43.258474,-43.433030,5.774779,6.139667,6.162500
0,600028.SH,2019-08-01,16743,5.25,5.27,5.21,5.23,5.23,616872.58,-0.030634,5.362514,5.179486,42.928615,-77.147867,22.469955,5.324333,5.357333
0,600030.SH,2019-08-01,16744,23.09,23.19,22.81,22.92,22.92,809710.56,0.143135,23.703768,22.720232,51.752238,-102.582689,11.905549,23.385667,21.918333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,600276.SH,2020-01-02,18295,88.00,88.10,86.85,87.68,87.68,210809.91,-0.345742,87.726881,82.918119,53.727892,55.084385,2.193577,86.140333,87.454167
103,600309.SH,2020-01-02,18296,56.50,57.60,55.69,55.89,55.89,213379.65,1.824002,56.464963,49.288037,67.318349,154.362552,50.898752,51.418667,48.306667
103,600519.SH,2020-01-02,18297,1128.00,1145.06,1116.00,1130.00,1130.00,148099.16,-3.944016,1188.194602,1121.743398,46.814398,-98.839848,24.496850,1158.660667,1175.338000
103,600547.SH,2020-01-02,18298,32.31,32.96,32.26,32.90,32.90,489106.37,0.348359,32.722690,30.612310,52.089075,155.084385,32.165604,31.288667,31.498333


In [38]:
df_account_value = df_account_value.rename(columns={'date': 'time'})

trade = trade.rename(columns={'date': 'time'})
plot_end_date = '2019-12-31'
trade_plot = trade[trade['time'] <= plot_end_date]
trade_plot


,tic,time,index,open,high,low,close,adjusted_close,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,600000.SH,2019-08-01,16740,11.76,11.85,11.63,11.65,11.65,324717.02,0.061723,11.954291,11.217709,51.629338,30.105973,9.109642,11.632667,11.516667
0,600009.SH,2019-08-01,16741,82.78,83.46,81.60,82.01,82.01,72583.66,1.456486,85.100887,78.654113,57.259597,28.697822,13.903367,81.707000,76.533333
0,600016.SH,2019-08-01,16742,6.04,6.06,6.01,6.02,6.02,517116.83,-0.045649,6.088116,5.935884,43.258474,-43.433030,5.774779,6.139667,6.162500
0,600028.SH,2019-08-01,16743,5.25,5.27,5.21,5.23,5.23,616872.58,-0.030634,5.362514,5.179486,42.928615,-77.147867,22.469955,5.324333,5.357333
0,600030.SH,2019-08-01,16744,23.09,23.19,22.81,22.92,22.92,809710.56,0.143135,23.703768,22.720232,51.752238,-102.582689,11.905549,23.385667,21.918333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,600276.SH,2019-12-31,18280,85.27,88.18,85.27,87.52,87.52,220793.14,-0.573297,87.363472,83.078528,53.502459,20.134853,2.193577,86.384333,87.304333
102,600309.SH,2019-12-31,18281,56.00,57.19,55.68,56.17,56.17,142068.42,1.780555,56.261337,48.778663,68.325632,161.334644,48.804471,51.089000,48.101667
102,600519.SH,2019-12-31,18282,1183.00,1188.00,1176.51,1183.00,1183.00,22588.81,-1.622947,1189.173694,1119.997306,53.683567,62.426718,3.403761,1162.119000,1175.618167
102,600547.SH,2019-12-31,18283,32.44,32.79,32.22,32.62,32.62,422904.08,0.286032,32.557092,30.709908,51.090729,150.027238,29.847459,31.208333,31.537833


In [39]:
plotter = ReturnPlotter(df_account_value, trade_plot, TRADE_START_DATE, plot_end_date)
plotter.plot()

**Error: ValueError: x and y must have same first dimension, but have shapes (103,) and (104,)**

https://github.com/AI4Finance-Foundation/FinRL-Tutorials/issues/91

df_account_value 知道2019-12-31，而time_series却到了2020-01-02， 所以plot中传入的end_date需要变成2020-01-01，同时需要移除trade中2020-01-02的数据

In [53]:
# ticket: SSE 50：000016
plotter.plot("000016")

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found


OSError: 获取失败，请检查网络.

### CSI 300

In [76]:
baseline_df = plotter.get_baseline('399300')
baseline_df

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found


OSError: 获取失败，请检查网络.

In [78]:
daily_return = plotter.get_return(df_account_value)
daily_return

time
2019-08-01 00:00:00+00:00         NaN
2019-08-02 00:00:00+00:00   -0.002544
2019-08-05 00:00:00+00:00   -0.003112
2019-08-06 00:00:00+00:00   -0.000487
2019-08-07 00:00:00+00:00   -0.002620
                               ...   
2019-12-25 00:00:00+00:00   -0.005259
2019-12-26 00:00:00+00:00    0.011977
2019-12-27 00:00:00+00:00   -0.006358
2019-12-30 00:00:00+00:00    0.035400
2019-12-31 00:00:00+00:00    0.001424
Name: daily_return, Length: 103, dtype: float64

In [79]:
daily_return_base = plotter.get_return(baseline_df, value_col_name="close")
daily_return_base

NameError: name 'baseline_df' is not defined

In [80]:
perf_func = timeseries.perf_stats
perf_stats_all = perf_func(returns=daily_return, positions=None, transactions=None, turnover_denom="AGB")
print("==============DRL Strategy Stats===========")
print(f"perf_stats_all: {perf_stats_all}")

==============DRL Strategy Stats===========
perf_stats_all: Annual return          0.250536
Cumulative returns     0.095686
Annual volatility      0.166416
Sharpe ratio           1.439246
Calmar ratio           2.616460
Stability              0.005206
Max drawdown          -0.095754
Omega ratio            1.272748
Sortino ratio          2.399210
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.281685
Daily value at risk   -0.020016
dtype: float64
